In [41]:
import pandas as pd
from collections import Counter
import csv
# import library yang di butuhkan

df = pd.read_csv('test-revisi.csv', on_bad_lines='skip')  #membaca file csv nya 
text = []
label = []
print(df)

                                                   Teks  label
0     [PROMO] Beli paket Flash mulai 1GB di MY TELKO...      1
1     2.5 GB/30 hari hanya Rp 35 Ribu Spesial buat A...      1
2     2016-07-08 11:47:11.Plg Yth, sisa kuota Flash ...      1
3     2016-08-07 11:29:47.Plg Yth, sisa kuota Flash ...      1
4     4.5GB/30 hari hanya Rp 55 Ribu Spesial buat an...      1
...                                                 ...    ...
7231  7) untuk tahu kapan jadwal sidang kamu cukup k...      1
7232  contoh: iteung minta jadwal sidang tipe bimbin...      1
7233  Sekedar mengingatkan untuk tahun itu formatnya...      1
7234                           Contohnya liat di poin 6      1
7235                                 To be continued...      1

[7236 rows x 2 columns]


In [42]:
with open('test-revisi.csv', newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        text.append(row[0])
        label.append(row[0]) 

In [43]:
# import re

# def casefolding(text):
#     text = text.lower() # merubah bentuk menjadi lower case
#     text = re.sub(r'\d+', '', text) # menghapus angka
#     text = re.sub(r'[^\w\s]','',text) # menghapus karakter tanda baca
#     text = text.strip()
#     return text
# key_norm = pd.read_csv('test-revisi.csv', on_bad_lines='skip')

# def text_normalize(text):
#     text = ' '.join([key_norm[key_norm['halo'] == word]['hasil'].values[0] if (key_norm["singkat"] == word).any() else word for word in text.split()])
#     text = str.lower(text)
#     return text

In [44]:
# referensi labeling gagal

# from tortus import Tortus
# tortus = Tortus(df, id_column=10, annotations=None, random=True, labels=['a', 'b', 'c'])

In [45]:
print("jumlah data:{}".format(len(text)))  #mengidentifikasi dan mengetahui seberapa panjang jumlah item pada objek
print(Counter(df.columns))

jumlah data:7237
Counter({'Teks': 1, 'label': 1})


In [46]:
# nltk stopword

import nltk
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
text[-1] = label[-1].format(lambda x: [item for item in x if item not in stop_words])

In [47]:
# stem

from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

data="nanti ketemu dimana?"
# data = pd.read_csv('test-copy.csv')
words = word_tokenize(data)
ps = PorterStemmer()
for w in words:
    rootWord=ps.stem(w)
    print(rootWord)

nanti
ketemu
dimana
?


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AKU\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
import pandas as pd
stem_word = pd.read_csv('test-revisi.csv', on_bad_lines='skip')
stem_word = stem_word.sort_values(by=['Teks'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(stem_word['Teks'], stem_word['label'], test_size=0.2, random_state=111)



print("Data training:")
print(len(X_train))
#
print("Data testing:")
print(len(X_test))

Data training:
5788
Data testing:
1448


In [53]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [54]:
import numpy as np
pred = text_clf.predict(X_test)
akurasi = np.mean(pred==y_test)
print("Akurasi: {}".format(akurasi))

Akurasi: 1.0


In [55]:
sms_baru = ['silahkan transfer']
pred = text_clf.predict(sms_baru)
print("Hasil prediksi {}".format(pred))

Hasil prediksi [1]
